<br>
<center><font size = 15> Coordinate descent algorithm을 이용한 LASSO 추정 </font></center>
<br>

<img  height = 2000px src = "./img/lasso_coor.PNG">

<br>
<br>



<div style = "width : 400px; paddig-right:10px; float : right;">
    <font size = 8>
        190524 이성령
    </font></div>

# Library Load

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import  Lasso , LassoCV
from sklearn.metrics import mean_squared_error
import pandas as pd , re
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import warnings
np.random.seed(19510)

## Functions

In [2]:
def random_uniform(size) :
    _  = np.random.uniform(low = -1 , high = 1 , size = [size])[:,np.newaxis]
    return _

def noise(size) :
    _  = np.random.normal(loc = 0 , scale = 1 , size = [size])[:,np.newaxis]
    return _

def generate_x(size) :
    total = random_uniform(size)
    for _ in np.arange(19) :
        column = random_uniform(size)
        total = np.concatenate( (total , column) , axis = 1)
    return total

def plotting(data , title , lambda_list , ax ) :
    for idx , i in enumerate(data.columns.tolist()) :
        ax.plot(np.array(lambda_list) , data.loc[:,i].values , label = str(idx))
    ax.legend(frameon=False, loc='upper center', ncol=10)
    ax.set_title(title, color ="black", fontsize = 20)
    
def Loop_func(lambda_list , f , train_x , train_y , test_x , test_y ) :
    train_mse  , test_mse = [] , []
    coef = pd.DataFrame()
    for i in lambda_list :
        func = f(alpha=i)
        func.fit(train_x , train_y )
        string = str(func)
        if re.split(r"[(]", string)[0] == "Ridge" :
            func_coef = pd.DataFrame(func.coef_)
        else :
            func_coef = pd.DataFrame(func.coef_).T
        coef = pd.concat([ coef , func_coef ], axis = 0)
        func_train_mse = mean_squared_error(train_y , func.predict(train_x) )
        func_test_mse = mean_squared_error(test_y , func.predict(test_x) )
        train_mse.append(func_train_mse)
        test_mse.append(func_test_mse)
    coef.reset_index(drop= True , inplace= True)
    return train_mse  , test_mse , coef

def loss_plot(x , y , label , ax ) :
    ax.plot(x,y, label = label)
    ax.legend(fontsize = 15)
    label2 = re.split(" " , label)
    ax.set_title( label2[0] , fontsize = 30)
    arg_min = np.argmin(y)
    ax.text(x[arg_min] , y[arg_min], r"{} mse : {:.3f} , lambda : {:.5f}".format(label2[1] , y[arg_min] , x[arg_min]) , fontsize = 15)

## Data Generate

In [3]:
train_n = 100
train_x = generate_x(train_n)
test_n = 10000
test_x = generate_x(test_n)

# Case1

## $\beta_1 = \beta_2 = 1 , $
## $\beta_3 = \beta_4 = ... = \beta_{20} = 0$

In [4]:
beta = np.array([1] * 2 + [0] * 18)[ :, np.newaxis ]
train_eps = noise(train_n)
test_eps = noise(test_n)
train_y = train_x.dot(beta) + train_eps
test_y  = test_x.dot(beta) + test_eps
##

Lasso_lambda_list = np.linspace(1e-5 , 0.35 , 100)

lasso_train_mse  , lasso_test_mse , lasso_coef = Loop_func(Lasso_lambda_list , Lasso, train_x , train_y, test_x , test_y)


## Lassocv를 통해 K=10 일 때의 최적의 alpha 구하기

## 결국 구해야 하는 것과 알아야 하는 것
* lambda 는 결국 cv를 통해서 얻은 최적값
* j번째 coef 값
* j번째 x와 y간의 corr
* 한개씩 업데이트를 하는데, 자기 자신을 제외한 나머지들의 correlation들의 합

In [5]:
np.random.seed(19510)
reg = LassoCV(cv=5, random_state=0).fit(train_x , train_y )
best_alpha = reg.alpha_
x_corr = pd.DataFrame(train_x).corr().values
print("Best Alpha : {}".format(best_alpha))
print("coefficient : {}".format(reg.coef_))

Best Alpha : 0.042029806612334804
coefficient : [ 0.82478003  0.7851741   0.          0.10372369 -0.          0.10283864
 -0.          0.13194043  0.         -0.         -0.         -0.2538367
  0.          0.         -0.         -0.         -0.          0.21818703
  0.         -0.11857493]


/root/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:1094: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [6]:
TEST = Lasso(alpha=best_alpha).fit(train_x, train_y)
TEST.coef_

array([ 0.82478003,  0.7851741 ,  0.        ,  0.10372369, -0.        ,
        0.10283864, -0.        ,  0.13194043,  0.        , -0.        ,
       -0.        , -0.2538367 ,  0.        ,  0.        , -0.        ,
       -0.        , -0.        ,  0.21818703,  0.        , -0.11857493])

## 3. Lasso Problem

<font size = 5> $\sum_{j=1}^{p}(y_j - \beta_j)^2 + \lambda \sum_{j=1}^{p}|\beta_j|$ </font >

<font size = 4; color = "green"  > $if\;\; \beta > 0\,\,, $ </font >

$\;\; \sum_{j=1}^{p}(y_j - \beta_j)^2 + \lambda \sum_{j=1}^{p}\beta_j$

$\;\; \sum (\beta^2-(y-\frac{\lambda}{2}))^2 + y^2-(y-\frac{\lambda}{2})^2) $

$\;\; (\beta - y + \frac{\lambda}{2})^2 + \lambda y - \frac{\lambda^2}{4}$

$\;\; min\hat\beta =  y-\frac{\lambda}{2} (\beta>0)  \qquad  y-\frac{\lambda}{2} > 0 $ 



<font size = 4; color = "green"  > $if\;\; \beta < 0\,\,, $  </font >

$\;\; \sum_{j=1}^{p}(y_j - \beta_j)^2 - \lambda \sum_{j=1}^{p}\beta_j$

$\;\; \sum (\beta^2-(y+\frac{\lambda}{2}))^2 + y^2-(y+\frac{\lambda}{2})^2) $

$\;\; (\beta - y - \frac{\lambda}{2})^2 - \lambda y - \frac{\lambda^2}{4}$

$\;\; min\hat\beta =  y + \frac{\lambda}{2} (\beta>0)  \qquad  y+ \frac{\lambda}{2} < 0 $ 


> <font size = 5; color = "red"  > Soft-Thresholding </font >

* wavelet shrinkage이라고 불린다고 하고 합니다.
* image-denoising에서 이 경우를 적용하면, 강조하고 싶은 부분을 강조해주는 효과가 있다고 합니다.

$\hat\beta_j^L=
\begin{cases}
y_j-\frac{\lambda}{2} , & \mbox{if } y_j > \frac{\lambda}{2} \\
y_j+\frac{\lambda}{2}, & \mbox{if }y_j < - \frac{\lambda}{2} \\
0 , & \mbox{if } |y_j| \le \frac{\lambda}{2} \\
\end{cases}
= \begin{cases}
\hat\beta_j-\frac{\lambda}{2} , & \mbox{if } \hat\beta_j > \frac{\lambda}{2} \\
\hat\beta_j+\frac{\lambda}{2}, & \mbox{if }\hat\beta_j< -\frac{\lambda}{2} \\
0 , & \mbox{if } |\hat\beta_j| \le \frac{\lambda}{2} \\
\end{cases}
$



$\hat\beta = sign(y)(|y|-\frac{\lambda}{2})_+$

$(|y|-\frac{\lambda}{2})_+=
\begin{cases}
|y_j|-\frac{\lambda}{2} , & \mbox{if } |y_j| > \frac{\lambda}{2} \\
0 , & \mbox{if } |y_j| < - \frac{\lambda}{2} \\
\end{cases}
$


$y = r_{j,y}-\sum_{l \neq j }\alpha_jr_{l,j}$ 

$r_{i,j} = corr(x_i,x_j) $

In [8]:
## init 값
np.set_printoptions(precision= 10000)
from scipy.spatial.distance import euclidean
coef = np.random.uniform(1,2,20)
x_corr = pd.DataFrame(train_x).corr().values
rang = np.arange(20)
tol = 1e-300
max_num = 1000
num = 0
dist_list = []
while True :
    pre_coef = np.copy(coef)
    coef     = np.copy(coef)
    for idx in rang :
        select_x = train_x[:,idx]
        r_xy = np.corrcoef(select_x , np.squeeze(train_y))[0,1]
        select_xx = np.where(rang != idx )[0]
        r_xx = np.sum(x_corr[idx,:][select_xx] * coef[select_xx])
        t = r_xy - r_xx
        term = np.abs(t) - best_alpha/2
        term = term if term > 0 else 0
        coef[idx] = np.sign(t)*term
        dist = euclidean(pre_coef , coef)
    if (dist < tol) | (num > max_num)  :
        print("Iteration : {} ,".format(num) )
        print("Dist : ", dist)
        print("Coordinate Descent : \n {} ".format(coef) )
        break
    else :
        num += 1 
        dist_list.append(dist)
        coef = np.copy(coef)
print("Best coefficient : \n {}".format(reg.coef_))


Iteration : 32 ,
Dist :  0.0
Coordinate Descent : 
 [ 0.4384162766998976    0.4221386154610102    0.0390262975748355
  0.09356878063212302  -0.00638351765772854   0.11588841496624547
 -0.04315549890425299   0.1393691004995831    0.
 -0.                   -0.                   -0.17805698575629736
  0.                    0.                   -0.04503708527489583
  0.                   -0.007894085619709959  0.11091489026565637
  0.039372944144943606 -0.11505485416195504 ] 
Best coefficient : 
 [ 0.8247800332329711   0.7851740973085213   0.
  0.10372369042089918 -0.                   0.10283863521387279
 -0.                   0.13194042689094157  0.
 -0.                  -0.                  -0.2538367013971219
  0.                   0.                  -0.
 -0.                  -0.                   0.2181870288488784
  0.                  -0.11857493091514436]


In [9]:

data = [
    go.Bar(
        x = list(rang) , 
        y = reg.coef_  ,
        marker = dict(
          color='rgba(55, 128, 191, 0.7)',
        ),
        name = 'Cross Validatiaon Coef'
    ),
    go.Bar(
        x = list(rang),
        y = coef ,
        base = 0,
        marker = dict(
          color='rgba(219, 64, 82, 0.7)',
        ),
        name = 'Coordinate Desecnt Coef'
    )
]

updatemenus = list([
    dict(type="buttons",
         active=-1,
         buttons=list([
            dict(label = 'CoordinateDeScent',
                 method = 'update',
                 args = [{'visible': [True,False]},
                         {'title': 'Coordinate Descent Coef'}]),
            dict(label = 'CrossValidation',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'CrossValidation Coef'}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True,  True]},
                         {'title': 'Coefficeint [CrossValidation and Coordinate Descent]'}])
        ]),
    )
])

layout = dict(title='Coefficeint [CrossValidation and Coordinate Descent]', showlegend=True,
              updatemenus=updatemenus)


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='base-bar')